Benjamin Luo, 20890448, b33luo@uwaterloo.ca

__SYDE 780 Course Project - Preprocessing__

<hr />

# ___BRFSS Dataset Preprocessing___

<hr />

## Data Dictionary

In [1]:
from bs4 import BeautifulSoup

data_dict_html = "./data/codebook.htm"

with open(data_dict_html, 'r', encoding='cp1252') as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')

In [2]:
import re
import pandas as pd

table_headers = soup.find_all('td', class_='l m linecontent')

data = []

for table_header in table_headers:

    text_content = table_header.get_text(separator=' ').replace("\xa0", " ")
    regex = re.split(r"Label: |Section Name: |Type of Variable: |SAS Variable Name: |Question Prologue: |Question:", text_content)
    label = regex[1].strip()#.lower().replace(" ", "_")
    data_type = regex[3].strip()
    sas_variable_name = regex[4].strip()
    question = regex[-1].strip()
    
    data.append({
        'Data Element Name': sas_variable_name,
        'Format': data_type,
        # 'Allowable Values': None,
        'Description': label,
        # 'Example': None,
        'Survey Question': question
    })

dd22 = pd.DataFrame(data).set_index('Data Element Name')

In [3]:
def remove_notes(text):
    notes_index = text.find('Notes:')
    if notes_index != -1:
        return text[:notes_index]
    else:
        return text
    
def remove_goto(text):
    goto_index = text.find('—Go to')
    if goto_index != -1:
        return text[:goto_index]
    else:
        return text

In [4]:
# Possible values
mappings = []
examples_num = []
examples_char = []

for table in soup.find_all('table', class_="table")[1:]:
    mapping = {}
    for row in table.find_all('tr')[2:]:
        cells = row.find_all('td')
        value = cells[0].text.strip()
        label = remove_goto(remove_notes(cells[1].text.strip()))
        mapping[value] = label
    mappings.append(mapping)
    examples_num.append(list(mapping.keys())[0])
    examples_char.append(list(mapping.values())[0])
    
dd22['Possible values'] = mappings
dd22['Example Num'] = examples_num
dd22['Example Char'] = examples_char

In [5]:
dd22.head(3)

,Format,Description,Survey Question,Possible values,Example Num,Example Char
Data Element Name,,,,,,
_STATE,Num,State FIPS Code,State FIPS Code,"{'1': 'Alabama', '2': 'Alaska', '4': 'Arizona'...",1,Alabama
FMONTH,Num,File Month,File Month,"{'1': 'January', '2': 'February', '3': 'March'...",1,January
IDATE,Char,Interview Date,Interview Date,{'HIDDEN': 'Data not displayed'},HIDDEN,Data not displayed


## Dataset

In [6]:
import pandas as pd

df22 = pd.read_sas('./data/LLCP2022.XPT', format='xport')
df22.shape

(445132, 328)

In [7]:
# Remove undocumented features
unique_values = {col: df22[col].unique().tolist() for col in df22.columns}
undoc_keys = set(df22.columns).difference(set(dd22.index.tolist()))
df22.drop(columns=undoc_keys, inplace=True)

assert df22.columns.tolist() == dd22.index.tolist()

In [12]:
"""I prompted AI to identify the relevant features from dd22[['Survey Question']].to_dict()

Independent variables
    CVDINFR4: (Ever told) you had a heart attack, also called a myocardial infarction?
    CVDCRHD4: (Ever told) (you had) angina or coronary heart disease?
    _MICHD: Respondents that have ever reported having coronary heart disease (CHD) or myocardial infarction (MI).

Demographic and socioeconomic
    _STATE: State of residence (geographic location can influence access to healthcare and environmental factors).
    SEXVAR: Sex of the respondent (cardiovascular risk varies by sex).
    _AGEG5YR: Age group (cardiovascular risk increases with age).
    _RACEGR4: Race/ethnicity (certain racial/ethnic groups have higher cardiovascular risk).
    _EDUCAG: Education level (lower education levels are associated with higher cardiovascular risk).
    _INCOMG1: Income level (lower income is associated with higher cardiovascular risk).
    MARITAL: Marital status (social support can influence cardiovascular health).
    EMPLOY1: Employment status (stress and job type can impact cardiovascular health).

Behavioral and lifestyle
    SMOKE100: Ever smoked at least 100 cigarettes (smoking is a major risk factor for cardiovascular disease).
    SMOKDAY2: Current smoking status (everyday, some days, or not at all).
    USENOW3: Current use of chewing tobacco, snuff, or snus.
    ECIGNOW2: Current use of e-cigarettes or vaping products.
    ALCDAY4: Days per month with alcohol consumption (excessive alcohol use can increase cardiovascular risk).
    AVEDRNK3: Average number of drinks per day when drinking.
    DRNK3GE5: Frequency of binge drinking (5+ drinks for men, 4+ for women).
    EXERANY2: Participation in physical activity or exercise (regular exercise reduces cardiovascular risk).
    SLEPTIM1: Average hours of sleep per night (poor sleep is linked to cardiovascular issues).
    MARIJAN1: Marijuana use (emerging evidence suggests potential cardiovascular risks).

Medical history and health status
    GENHLTH: Self-reported general health (poor general health is associated with higher cardiovascular risk).
    PHYSHLTH: Days in the past 30 days with poor physical health.
    MENTHLTH: Days in the past 30 days with poor mental health (stress and depression are linked to cardiovascular disease).
    DIABETE4: Ever told you had diabetes (diabetes is a major risk factor for cardiovascular disease).
    PREDIAB2: Ever told you had prediabetes or borderline diabetes.
    HAVARTH4: Ever told you had arthritis (chronic inflammation can impact cardiovascular health).
    CHCCOPD3: Ever told you had COPD, emphysema, or chronic bronchitis (lung disease can strain the cardiovascular system).
    ADDEPEV3: Ever told you had a depressive disorder (depression is linked to cardiovascular disease).
    ASTHMA3: Ever told you had asthma (chronic respiratory conditions can impact cardiovascular health).
    TOLDCFS: Ever told you had Chronic Fatigue Syndrome (chronic conditions can strain the cardiovascular system).

Diet and weight-related factors
    WEIGHT2: Self-reported weight (obesity is a major risk factor for cardiovascular disease).
    HEIGHT3: Self-reported height (used to calculate BMI).
    _BMI5: Body Mass Index (BMI) (overweight and obesity increase cardiovascular risk).
    _BMI5CAT: BMI categories (underweight, normal, overweight, obese).
    FOODSTMP: Received food stamps (proxy for food insecurity, which can impact diet quality).
    SDHFOOD1: Frequency of food insecurity (poor nutrition can increase cardiovascular risk).
    
Healthcare access and utilization
    PRIMINSR: Primary source of health insurance (lack of insurance can limit access to preventive care).
    MEDCOST1: Could not see a doctor due to cost in the past 12 months (financial barriers to care).
    CHECKUP1: Time since last routine checkup (regular checkups can help manage cardiovascular risk).
    CHKHEMO3: Frequency of A1C checks (important for managing diabetes, a cardiovascular risk factor).
    EYEEXAM1: Time since last eye exam (diabetes-related eye exams can indicate cardiovascular risk).
    DIABEDU1: Time since last diabetes management class (education can improve cardiovascular outcomes).
    
Environmental and social
    SDHISOLT: Frequency of feeling socially isolated (social isolation is linked to cardiovascular risk).
    EMTSUPRT: Frequency of getting social and emotional support (social support can reduce cardiovascular risk).
    SDHSTRE1: Frequency of stress (chronic stress is a risk factor for cardiovascular disease).
    SDHTRNSP: Lack of reliable transportation (can limit access to healthcare and healthy food options).
    SDHBILLS: Inability to pay bills (financial stress can impact cardiovascular health).
    
Other
    COVIDPOS: Ever tested positive for COVID-19 (COVID-19 can have long-term cardiovascular effects).
    COVIDSMP: Long-term COVID symptoms (can strain the cardiovascular system).
    FLUSHOT7: Received a flu shot in the past year (preventing infections can reduce cardiovascular strain).
    PNEUVAC4: Ever had a pneumonia shot (preventing infections can reduce cardiovascular strain).
    TETANUS1: Received a tetanus shot in the past 10 years (proxy for preventive healthcare access).
"""
pass

In [13]:
# Remove irrelevant features that do not affect cardiac health
relevant_features = ["CVDINFR4","CVDCRHD4","_MICHD","_STATE","SEXVAR","_AGEG5YR","_RACEGR4","_EDUCAG","_INCOMG1","MARITAL","EMPLOY1",
                     "SMOKE100","SMOKDAY2","USENOW3","ECIGNOW2","ALCDAY4","AVEDRNK3","DRNK3GE5","EXERANY2","SLEPTIM1","MARIJAN1","GENHLTH",
                     "PHYSHLTH","MENTHLTH","DIABETE4","PREDIAB2","HAVARTH4","CHCCOPD3","ADDEPEV3","ASTHMA3","TOLDCFS","WEIGHT2","HEIGHT3",
                     "_BMI5","_BMI5CAT","FOODSTMP","SDHFOOD1","PRIMINSR","MEDCOST1","CHECKUP1","CHKHEMO3","EYEEXAM1","DIABEDU1","SDHISOLT",
                     "EMTSUPRT","SDHSTRE1","SDHTRNSP","SDHBILLS","COVIDPOS","COVIDSMP","FLUSHOT7","PNEUVAC4","TETANUS1"]

dd22c = dd22.loc[relevant_features]
df22c = df22[relevant_features]

In [ ]:
# Remove features where the majority (>80%) of responses are NaN
df22c = df22c.dropna(axis=1, thresh=int(0.80 * len(df22)) + 1)
removed_cols = df22.columns.difference(df22c.columns)
dd22c = dd22c.loc[~dd22c.index.isin(removed_cols)]

assert dd22c.index.tolist() == df22c.columns.tolist()

In [15]:
dd22c['Example'] = df22c.dropna(how='any').iloc[0].values
# dd22c.drop(columns='Example', inplace=True)

dd22c = dd22c.loc[~dd22c.index.isin(removed_cols)]
dd22c.head(3)

,Format,Description,Survey Question,Possible values,Example Num,Example Char,Example
Data Element Name,,,,,,,
CVDINFR4,Num,Ever Diagnosed with Heart Attack,"(Ever told) you had a heart attack, also calle...","{'1': 'Yes', '2': 'No', '7': 'Don’t know/Not s...",1,Yes,2.0
CVDCRHD4,Num,Ever Diagnosed with Angina or Coronary Heart D...,(Ever told) (you had) angina or coronary heart...,"{'1': 'Yes', '2': 'No', '7': 'Don’t know/Not s...",1,Yes,2.0
_MICHD,Num,Ever had CHD or MI,Respondents that have ever reported having cor...,"{'1': 'Reported having MI or CHD', '2': 'Did n...",1,Reported having MI or CHD,2.0


In [17]:
dd22c['Variable'] = ["heart_attack", "angina_or_chd", "chd_or_mi", "state", "sex", "age", 
                     "ethnicity", "education", "income", "marital", "employment", "smoker_past", 
                     "smoker_now", "vaper", "alcohol_days", "exercise_days", "sleep", "general_health", "physical_health_days",
                     "mental_health_days", "diabetes", "arthritis", "copd", "depression", "asthma", "weight", "height", "bmi", "bmi_category",
                     "health_insurance", "low_ses", "checkup_days", "covid", "flu_shot", "pneumonia_shot", "tetanus_shot"]

In [ ]:
# dd22c.to_csv('data_dictionary.csv')
# df22c.to_csv('dataset.csv')

In [19]:
print(dd22c['Possible values'].values)

[{'1': 'Yes', '2': 'No', '7': 'Don’t know/Not sure', '9': 'Refused', 'BLANK': 'Not asked or Missing'}
 {'1': 'Yes', '2': 'No', '7': 'Don’t know/Not sure', '9': 'Refused', 'BLANK': 'Not asked or Missing'}
 {'1': 'Reported having MI or CHD', '2': 'Did not report having MI or CHD', 'BLANK': 'Not asked or Missing'}
 {'1': 'Alabama', '2': 'Alaska', '4': 'Arizona', '5': 'Arkansas', '6': 'California', '8': 'Colorado', '9': 'Connecticut', '10': 'Delaware', '11': 'District of Columbia', '12': 'Florida', '13': 'Georgia', '15': 'Hawaii', '16': 'Idaho', '17': 'Illinois', '18': 'Indiana', '19': 'Iowa', '20': 'Kansas', '21': 'Kentucky', '22': 'Louisiana', '23': 'Maine', '24': 'Maryland', '25': 'Massachusetts', '26': 'Michigan', '27': 'Minnesota', '28': 'Mississippi', '29': 'Missouri', '30': 'Montana', '31': 'Nebraska', '32': 'Nevada', '33': 'New Hampshire', '34': 'New Jersey', '35': 'New Mexico', '36': 'New York', '37': 'North Carolina', '38': 'North Dakota', '39': 'Ohio', '40': 'Oklahoma', '41': 'O

### Decoding and refactoring features

In [20]:
import pandas as pd

df = pd.read_csv('data/dataset.csv')
dd = pd.read_csv('data/data_dictionary.csv')

In [21]:
refactored_names = {
    'CVDINFR4': 'mi',
    'CVDCRHD4': 'chd_angina',
    '_MICHD': 'chd_mi',
    '_STATE': 'state_fips_code',
    'SEXVAR': 'sex',
    '_AGEG5YR': 'age_group',
    '_RACEGR4': 'race_ethnicity',
    '_EDUCAG': 'education_level',
    '_INCOMG1': 'income_category',
    'MARITAL': 'marital_status',
    'EMPLOY1': 'employment_status',
    'SMOKE100': 'smoked_100_cigarettes',
    'USENOW3': 'uses_tobacco',
    'ECIGNOW2': 'uses_e_cigarettes',
    'ALCDAY4': 'days_drinking_alcohol',
    'EXERANY2': 'exercises_regularly',
    'SLEPTIM1': 'average_sleep_hours',
    'GENHLTH': 'general_health',
    'PHYSHLTH': 'physical_health_not_good_days',
    'MENTHLTH': 'mental_health_not_good_days',
    'DIABETE4': 'diabetes_diagnosis',
    'HAVARTH4': 'arthritis_diagnosis',
    'CHCCOPD3': 'copd_diagnosis',
    'ADDEPEV3': 'depression_diagnosis',
    'ASTHMA3': 'asthma_diagnosis',
    'WEIGHT2': 'weight',
    'HEIGHT3': 'height',
    '_BMI5': 'body_mass_index',
    '_BMI5CAT': 'body_mass_index_category',
    'PRIMINSR': 'primary_health_insurance',
    'MEDCOST1': 'could_not_afford_medical_care',
    'CHECKUP1': 'last_doctor_visit',
    'COVIDPOS': 'covid_positive_diagnosis',
    'FLUSHOT7': 'had_flu_vaccine',
    'PNEUVAC4': 'had_pneumonia_vaccine',
    'TETANUS1': 'had_tetanus_shot'
}
df = df.rename(columns=refactored_names).reset_index(drop=True)

In [22]:
dd['Renamed Variable'] = refactored_names.values()

In [23]:
dd[["Possible values", "Renamed Variable"]].to_csv("data/ddcrop.csv", index=False)

In [24]:
# Map values
shortened_values_list = [
    {'1': 'Yes', '2': 'No', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'},
    {'1': 'Yes', '2': 'No', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'},
    {'1': 'Yes', '2': 'No', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'},
    {'1': 'AL', '2': 'AK', '4': 'AZ', '5': 'AR', '6': 'CA', '8': 'CO', '9': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY', '66': 'GU', '72': 'PR', '78': 'VI'},
    {'1': 'Male', '2': 'Female'},
    {'1': '18-24', '2': '25-29', '3': '30-34', '4': '35-39', '5': '40-44', '6': '45-49', '7': '50-54', '8': '55-59', '9': '60-64', '10': '65-69', '11': '70-74', '12': '75-79', '13': '80+', '14': 'Unknown'},
    {'1': 'White', '2': 'Black', '3': 'Other', '4': 'Multiracial', '5': 'Hispanic', '9': 'Unknown', 'BLANK': 'Missing'},
    {'1': 'No HS', '2': 'HS Grad', '3': 'Some College', '4': 'College Grad', '9': 'Unknown'},
    {'1': '<$15K', '2': '$15K-25K', '3': '$25K-35K', '4': '$35K-50K', '5': '$50K-100K', '6': '$100K-200K', '7': '$200K+', '9': 'Unknown'},
    {'1': 'Married', '2': 'Divorced', '3': 'Widowed', '4': 'Separated', '5': 'Single', '6': 'Unmarried Partner', '9': 'Refused', 'BLANK': 'Missing'},
    {'1': 'Employed', '2': 'Self-Employed', '3': 'Unemployed >1yr', '4': 'Unemployed <1yr', '5': 'Homemaker', '6': 'Student', '7': 'Retired', '8': 'Disabled', '9': 'Refused', 'BLANK': 'Missing'},
    {'1': 'Yes', '2': 'No', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'},
    {'1': 'Daily', '2': 'Occasionally', '3': 'Never', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'},
    {'1': 'Never', '2': 'Daily', '3': 'Occasionally', '4': 'Former', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'},
    {'101 - 199': 'Days/Week', '201 - 299': 'Days/Month', '777': 'Unknown', '888': 'None', '999': 'Refused', 'BLANK': 'Missing'},
    {'1': 'Yes', '2': 'No', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'},
    {'1 - 24': 'Hours', '77': 'Unknown', '99': 'Refused', 'BLANK': 'Missing'},
    {'1': 'Excellent', '2': 'Very Good', '3': 'Good', '4': 'Fair', '5': 'Poor', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'},
    {'1 - 30': 'Days', '88': 'None', '77': 'Unknown', '99': 'Refused', 'BLANK': 'Missing'},
    {'1 - 30': 'Days', '88': 'None', '77': 'Unknown', '99': 'Refused', 'BLANK': 'Missing'},
    {'1': 'Yes', '2': 'Gestational', '3': 'No', '4': 'Prediabetes', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'},
    {'1': 'Yes', '2': 'No', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'},
    {'1': 'Yes', '2': 'No', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'},
    {'1': 'Yes', '2': 'No', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'},
    {'1': 'Yes', '2': 'No', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'},
    {'50 - 0776': 'Weight (lbs)', '7777': 'Unknown', '9023 - 9352': 'Weight (kg)', '9999': 'Refused', 'BLANK': 'Missing'},
    {'200 - 711': 'Height (ft/in)', '7777': 'Unknown', '9061 - 9998': 'Height (m/cm)', '9999': 'Refused', 'BLANK': 'Missing'},
    {'1 - 9999': '≥1', 'BLANK': 'Unknown'},
    {'1': 'Underweight', '2': 'Normal', '3': 'Overweight', '4': 'Obese', 'BLANK': 'Unknown'},
    {'1': 'Employer Plan', '2': 'Private Plan', '3': 'Medicare', '4': 'Medigap', '5': 'Medicaid', '6': 'CHIP', '7': 'Military', '8': 'IHS', '9': 'State Plan', '10': 'Other Govt', '88': 'No Coverage', '77': 'Unknown', '99': 'Refused', 'BLANK': 'Missing'},
    {'1': 'Yes', '2': 'No', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'},
    {'1': '<1yr', '2': '1-2yrs', '3': '2-5yrs', '4': '5+ yrs', '7': 'Unknown', '8': 'Never', '9': 'Refused', 'BLANK': 'Missing'},
    {'1': 'Yes', '2': 'No', '3': 'Home Test+', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'},
    {'1': 'Yes', '2': 'No', '7': 'Unknown', '9': 'Refused', 'BLANK': ''},
    {'1': 'Yes', '2': 'No', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'},
    {'1': 'Tdap', '2': 'Tetanus', '3': 'Unknown Type', '4': 'None', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'}
]

# Assign the shortened values back to the DataFrame
dd['Possible values'] = shortened_values_list

In [33]:
df.columns

Index(['Unnamed: 0', 'mi', 'chd_angina', 'chd_mi', 'state_fips_code', 'sex',
       'age_group', 'race_ethnicity', 'education_level', 'income_category',
       'marital_status', 'employment_status', 'smoked_100_cigarettes',
       'uses_tobacco', 'uses_e_cigarettes', 'days_drinking_alcohol',
       'exercises_regularly', 'average_sleep_hours', 'general_health',
       'physical_health_not_good_days', 'mental_health_not_good_days',
       'diabetes_diagnosis', 'arthritis_diagnosis', 'copd_diagnosis',
       'depression_diagnosis', 'asthma_diagnosis', 'weight', 'height',
       'body_mass_index', 'body_mass_index_category',
       'primary_health_insurance', 'could_not_afford_medical_care',
       'last_doctor_visit', 'covid_positive_diagnosis', 'had_flu_vaccine',
       'had_pneumonia_vaccine', 'had_tetanus_shot'],
      dtype='object')

In [40]:
# Define undesirable values based on the data dictionary
undesirable_values = {
    'heart_attack': [7, 9],
    'angina': [7, 9],
    'coronary_heart_disease': [],
    'state_fips_code': [],
    'sex': [],
    'age_group': [14],
    'race_ethnicity': [9],
    'education_level': [9],
    'income_category': [9],
    'marital_status': [9],
    'employment_status': [9],
    'smoked_100_cigarettes': [7, 9],
    'uses_tobacco': [7, 9],
    'uses_e_cigarettes': [7, 9],
    'days_drinking_alcohol': [777, 888, 999],
    'exercises_regularly': [7, 9],
    'average_sleep_hours': [77, 99],
    'general_health': [7, 9],
    'physical_health_not_good_days': [77, 99],
    'mental_health_not_good_days': [77, 99],
    'diabetes_diagnosis': [7, 9],
    'arthritis_diagnosis': [7, 9],
    'copd_diagnosis': [7, 9],
    'depression_diagnosis': [7, 9],
    'asthma_diagnosis': [7, 9],
    'weight': [7777, 9999],
    'height': [7777, 9999],
    'body_mass_index': [],
    'body_mass_index_category': [],
    'primary_health_insurance': [77, 99],
    'could_not_afford_medical_care': [7, 9],
    'last_doctor_visit': [7, 9],
    'covid_positive_diagnosis': [7, 9],
    'had_flu_vaccine': [7, 9],
    'had_pneumonia_vaccine': [7, 9],
    'had_tetanus_shot': [7, 9]
}

# Remove rows with undesirable values
# for column, values in undesirable_values.items():
#     df = df[~df[column].isin(values)]

In [41]:
# Variable transformation from strings to dictionaries
import json
import re

def replace_quotes(s):
    s = re.sub(r"(\w)'(\w)", r"\1%\2", s)
    s = s.replace("'", '"')
    s = s.replace("%", "'")
    
    return s

dd['Possible values'] = dd['Possible values'].astype(str).apply(replace_quotes).apply(json.loads)

In [42]:
dd['Possible values'].values[0]

{'1': 'Yes', '2': 'No', '7': 'Unknown', '9': 'Refused', 'BLANK': 'Missing'}

In [43]:
df = df.replace('nan', pd.NA).dropna()
df = df.dropna(how='all')
for column in df.columns:
    if column != "Unnamed: 0":
        print(df[column].unique())

[2. 1. 7. 9.]
[2. 1. 7. 9.]
[2. 1.]
[ 1.  2.  4.  5.  6.  8.  9. 10. 11. 12. 13. 15. 16. 17. 18. 19. 20. 21.
 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37. 38. 39.
 40. 41. 42. 44. 45. 46. 47. 48. 49. 50. 51. 53. 54. 55. 56. 66. 72. 78.]
[2. 1.]
[13.  8. 14.  5. 11. 10.  9. 12.  7.  6.  4.  3.  2.  1.]
[1. 2. 3. 4. 9. 5.]
[2. 4. 3. 1. 9.]
[3. 6. 9. 5. 4. 1. 7. 2.]
[3. 1. 2. 4. 5. 6. 9.]
[2. 7. 5. 8. 1. 4. 3. 6. 9.]
[2. 1. 7. 9.]
[3. 2. 1. 7. 9.]
[1. 4. 2. 3. 7. 9.]
[888. 203. 205. 201. 202. 204. 106. 105. 102. 101. 210. 230. 225. 229.
 103. 228. 104. 777. 999. 107. 207. 208. 206. 215. 220. 212. 218. 216.
 217. 209. 214. 221. 213. 227. 224. 226. 222. 223. 211. 219.]
[2. 1. 7. 9.]
[ 6.  5.  7.  9.  8.  4. 10.  1. 12. 77. 18.  3. 99.  2. 11. 15. 13. 16.
 14. 20. 23. 24. 19. 21. 17. 22.]
[1. 2. 4. 5. 3. 7. 9.]
[88.  2.  1.  8.  5. 30.  4. 23. 14. 77. 15.  3. 10.  7. 25. 99.  6. 21.
 20. 29. 16.  9. 27. 28. 12. 13. 11. 26. 17. 24. 18. 19. 22.]
[88.  3.  9.  5. 15. 20. 14. 

In [44]:
df.shape

(365242, 37)

In [45]:
mapping = dd[["Renamed Variable", "Possible values"]].set_index("Renamed Variable").to_dict(orient="index")
mapping

def map_values(feature_name, value):
    return mapping.get(feature_name, {}).get('Possible values', {}).get(str(int(value)), value)

for column in mapping.keys():
    df[column] = df[column].apply(lambda x: map_values(column, x))

In [46]:
df.head()

,Unnamed: 0,mi,chd_angina,chd_mi,state_fips_code,sex,age_group,race_ethnicity,education_level,income_category,...,height,body_mass_index,body_mass_index_category,primary_health_insurance,could_not_afford_medical_care,last_doctor_visit,covid_positive_diagnosis,had_flu_vaccine,had_pneumonia_vaccine,had_tetanus_shot
1,1,No,No,No,AL,Female,80+,White,HS Grad,$25K-35K,...,503.0,2657.0,Overweight,Medicare,No,Never,No,No,No,None
2,2,No,No,No,AL,Female,55-59,White,College Grad,$100K-200K,...,502.0,2561.0,Overweight,Employer Plan,No,<1yr,Yes,No,No,Unknown
3,3,No,No,No,AL,Female,Unknown,White,HS Grad,Unknown,...,505.0,2330.0,Normal,Refused,No,<1yr,No,Yes,Yes,None
4,4,No,No,No,AL,Female,40-44,White,Some College,$25K-35K,...,502.0,2177.0,Normal,Military,No,<1yr,No,No,Yes,None
5,5,Yes,No,Yes,AL,Male,80+,White,HS Grad,Unknown,...,511.0,2608.0,Overweight,Medicare,No,<1yr,No,No,Yes,None


In [47]:
dd

,Data Element Name,Format,Description,Survey Question,Possible values,Example Char,Example,Renamed Variable
0,CVDINFR4,Char,Ever Diagnosed with Heart Attack,"(Ever told) you had a heart attack, also calle...","{'1': 'Yes', '2': 'No', '7': 'Unknown', '9': '...",Yes,2,mi
1,CVDCRHD4,Char,Ever Diagnosed with Angina or Coronary Heart D...,(Ever told) (you had) angina or coronary heart...,"{'1': 'Yes', '2': 'No', '7': 'Unknown', '9': '...",Yes,2,chd_angina
2,_MICHD,Char,Ever had CHD or MI,Respondents that have ever reported having cor...,"{'1': 'Yes', '2': 'No', '7': 'Unknown', '9': '...",Reported having MI or CHD,2,chd_mi
3,_STATE,Char,State FIPS Code,State FIPS Code,"{'1': 'AL', '2': 'AK', '4': 'AZ', '5': 'AR', '...",Alabama,1,state_fips_code
4,SEXVAR,Char,Sex of Respondent,Sex of Respondent,"{'1': 'Male', '2': 'Female'}",Male,2,sex
5,_AGEG5YR,Char,Reported age in five-year age categories calcu...,Fourteen-level age category,"{'1': '18-24', '2': '25-29', '3': '30-34', '4'...",Age 18 to 24,13,age_group
6,_RACEGR4,Char,Computed Five level race/ethnicity category.,Five-level race/ethnicity category,"{'1': 'White', '2': 'Black', '3': 'Other', '4'...","White only, Non-Hispanic",1,race_ethnicity
7,_EDUCAG,Char,Computed level of education completed categories,Level of education completed,"{'1': 'No HS', '2': 'HS Grad', '3': 'Some Coll...",Did not graduate High School,2,education_level
8,_INCOMG1,Char,Computed income categories,Income categories,"{'1': '<$15K', '2': '$15K-25K', '3': '$25K-35K...","Less than $15,000",3,income_category
9,MARITAL,Char,Marital Status,Are you: (marital status),"{'1': 'Married', '2': 'Divorced', '3': 'Widowe...",Married,3,marital_status


In [48]:
for column in df.columns:
    if column != "Unnamed: 0":
        print(df[column].unique())

['No' 'Yes' 'Unknown' 'Refused']
['No' 'Yes' 'Unknown' 'Refused']
['No' 'Yes']
['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'DC' 'FL' 'GA' 'HI' 'ID' 'IL'
 'IN' 'IA' 'KS' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MN' 'MS' 'MO' 'MT' 'NE'
 'NV' 'NH' 'NJ' 'NM' 'NY' 'NC' 'ND' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VT' 'VA' 'WA' 'WV' 'WI' 'WY' 'GU' 'PR' 'VI']
['Female' 'Male']
['80+' '55-59' 'Unknown' '40-44' '70-74' '65-69' '60-64' '75-79' '50-54'
 '45-49' '35-39' '30-34' '25-29' '18-24']
['White' 'Black' 'Other' 'Multiracial' 'Unknown' 'Hispanic']
['HS Grad' 'College Grad' 'Some College' 'No HS' 'Unknown']
['$25K-35K' '$100K-200K' 'Unknown' '$50K-100K' '$35K-50K' '<$15K' '$200K+'
 '$15K-25K']
['Widowed' 'Married' 'Divorced' 'Separated' 'Single' 'Unmarried Partner'
 'Refused']
['Self-Employed' 'Retired' 'Homemaker' 'Disabled' 'Employed'
 'Unemployed <1yr' 'Unemployed >1yr' 'Student' 'Refused']
['No' 'Yes' 'Unknown' 'Refused']
['Never' 'Occasionally' 'Daily' 'Unknown' 'Refused']
['Never' 'Form

In [49]:
df = df.reset_index(drop=True)

In [50]:
df = df.drop(columns=["Unnamed: 0"])

In [53]:
import numpy as np

df = df.replace({'Unknown': np.nan, 'Refused': np.nan}).dropna(subset=['mi', 'chd_mi', 'chd_angina'])
df['cvd'] = df[['mi', 'chd_mi', 'chd_angina']].eq('Yes').any(axis=1).astype(int)
df = df.drop(columns=['mi', 'chd_angina', 'chd_mi'])

In [ ]:
df = df.drop(columns=['smoked_100_cigarettes', 'exercises_regularly', 'arthritis_diagnosis', 'body_mass_index_category', 'covid_positive_diagnosis', 'had_flu_vaccine', 'had_pneumonia_vaccine', 'had_tetanus_shot'])
df = df.drop(columns=['weight', 'height'])

In [67]:
df = df.replace('nan', pd.NA).dropna()
df = df.dropna(how='all')

In [ ]:
# Look for anything else to nitpick...
for column in df.columns:
    print(column, df[column].unique(), "\n")
    

state_fips_code ['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'DC' 'FL' 'GA' 'HI' 'ID' 'IL'
 'IN' 'IA' 'KS' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MN' 'MS' 'MO' 'MT' 'NE'
 'NV' 'NH' 'NJ' 'NM' 'NY' 'NC' 'ND' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VT' 'VA' 'WA' 'WV' 'WI' 'WY' 'GU' 'PR' 'VI'] 

sex ['Female' 'Male'] 

age_group ['80+' '55-59' '40-44' '70-74' '65-69' '60-64' '75-79' '50-54' '45-49'
 '35-39' '30-34' '25-29' '18-24'] 

race_ethnicity ['White' 'Black' 'Multiracial' 'Other' 'Hispanic'] 

education_level ['HS Grad' 'College Grad' 'Some College' 'No HS'] 

income_category ['$25K-35K' '$100K-200K' '$50K-100K' '$35K-50K' '<$15K' '$15K-25K'
 '$200K+'] 

marital_status ['Widowed' 'Married' 'Divorced' 'Separated' 'Unmarried Partner' 'Single'] 

employment_status ['Self-Employed' 'Retired' 'Homemaker' 'Disabled' 'Employed'
 'Unemployed <1yr' 'Unemployed >1yr' 'Student'] 

uses_tobacco ['Never' 'Occasionally' 'Daily'] 

uses_e_cigarettes ['Never' 'Former' 'Daily' 'Occasionally'] 

days

In [ ]:
"""
days_drinking_alcohol ['None' 203.0 205.0 201.0 202.0 204.0 106.0 101.0 210.0 230.0 102.0 225.0
 229.0 228.0 105.0 103.0 104.0 107.0 208.0 206.0 207.0 220.0 215.0 212.0
 216.0 217.0 209.0 214.0 221.0 218.0 213.0 227.0 226.0 222.0 224.0 223.0
 211.0 219.0] 

average_sleep_hours [ 6.  5.  9.  7.  8.  4. 10.  1. 18. 12.  3.  2. 11. 13. 15. 16. 14. 23.
 20. 24. 17. 22. 19.] 
 
physical_health_not_good_days ['None' 2.0 1.0 8.0 5.0 30.0 4.0 23.0 15.0 3.0 10.0 7.0 14.0 25.0 21.0
 20.0 29.0 9.0 6.0 27.0 13.0 12.0 28.0 26.0 17.0 16.0 24.0 11.0 18.0 19.0
 22.0] 

mental_health_not_good_days ['None' 3.0 9.0 5.0 15.0 10.0 18.0 1.0 2.0 4.0 30.0 7.0 20.0 6.0 14.0 8.0
 25.0 27.0 12.0 13.0 21.0 28.0 29.0 26.0 17.0 11.0 16.0 22.0 19.0 24.0
 23.0] 
 
body_mass_index [2657. 2561. 2177. ... 2839. 1702. 1735.] 

"""

In [74]:
dd[dd['Renamed Variable'].isin(['days_drinking_alcohol', 'average_sleep_hours', 'physical_health_not_good_days', 'mental_health_not_good_days', 'body_mass_index'])][['Renamed Variable', 'Possible values']].values

array([['days_drinking_alcohol',
        {'101 - 199': 'Days/Week', '201 - 299': 'Days/Month', '777': 'Unknown', '888': 'None', '999': 'Refused', 'BLANK': 'Missing'}],
       ['average_sleep_hours',
        {'1 - 24': 'Hours', '77': 'Unknown', '99': 'Refused', 'BLANK': 'Missing'}],
       ['physical_health_not_good_days',
        {'1 - 30': 'Days', '88': 'None', '77': 'Unknown', '99': 'Refused', 'BLANK': 'Missing'}],
       ['mental_health_not_good_days',
        {'1 - 30': 'Days', '88': 'None', '77': 'Unknown', '99': 'Refused', 'BLANK': 'Missing'}],
       ['body_mass_index', {'1 - 9999': '≥1', 'BLANK': 'Unknown'}]],
      dtype=object)

In [ ]:
df['physical_health_not_good_days'] = df['physical_health_not_good_days'].replace('None', 0).astype(int)
df['mental_health_not_good_days'] = df['mental_health_not_good_days'].replace('None', 0).astype(int)
df['body_mass_index'] = df['body_mass_index'] / 100
df['average_sleep_hours'] = df['average_sleep_hours'].astype(int)

/var/folders/l8/p53ms28n1791d89nx84s38x00000gn/T/ipykernel_84922/1797638999.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['physical_health_not_good_days'] = df['physical_health_not_good_days'].replace('None', 0).astype(int)
/var/folders/l8/p53ms28n1791d89nx84s38x00000gn/T/ipykernel_84922/1797638999.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['mental_health_not_good_days'] = df['mental_health_not_good_days'].replace('None', 0).astype(int)


In [79]:
def transform_value(x):
    x_str = str(int(x))
    if x_str.startswith('2'):
        return int(x_str[1:3])
    elif x_str.startswith('1'):
        return int(x_str[1:3]) * 4

df['days_drinking_alcohol'] = df['days_drinking_alcohol'].replace('None', 0).astype(int)
df['days_drinking_alcohol'] = df['days_drinking_alcohol'].apply(transform_value)

/var/folders/l8/p53ms28n1791d89nx84s38x00000gn/T/ipykernel_84922/2661526958.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['days_drinking_alcohol'] = df['days_drinking_alcohol'].replace('None', 0).astype(int)


In [82]:
df['days_drinking_alcohol'] = df['days_drinking_alcohol'].replace(np.nan, 0).astype(int)

In [88]:
for column in df.columns:
    print("-", column, df[column].unique())

- state_fips_code ['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'DC' 'FL' 'GA' 'HI' 'ID' 'IL'
 'IN' 'IA' 'KS' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MN' 'MS' 'MO' 'MT' 'NE'
 'NV' 'NH' 'NJ' 'NM' 'NY' 'NC' 'ND' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VT' 'VA' 'WA' 'WV' 'WI' 'WY' 'GU' 'PR' 'VI']
- sex ['Female' 'Male']
- age_group ['80+' '55-59' '40-44' '70-74' '65-69' '60-64' '75-79' '50-54' '45-49'
 '35-39' '30-34' '25-29' '18-24']
- race_ethnicity ['White' 'Black' 'Multiracial' 'Other' 'Hispanic']
- education_level ['HS Grad' 'College Grad' 'Some College' 'No HS']
- income_category ['$25K-35K' '$100K-200K' '$50K-100K' '$35K-50K' '<$15K' '$15K-25K'
 '$200K+']
- marital_status ['Widowed' 'Married' 'Divorced' 'Separated' 'Unmarried Partner' 'Single']
- employment_status ['Self-Employed' 'Retired' 'Homemaker' 'Disabled' 'Employed'
 'Unemployed <1yr' 'Unemployed >1yr' 'Student']
- uses_tobacco ['Never' 'Occasionally' 'Daily']
- uses_e_cigarettes ['Never' 'Former' 'Daily' 'Occasionally']
- da

In [91]:
unique_values_df = pd.DataFrame({
    'column_name': df.columns,
    'unique_values': [df[col].unique().tolist() for col in df.columns]
})
unique_values_df.to_csv("data/unique_values.csv", index=False)

In [84]:
df.to_csv("data/data_v5.csv", index=False)

In [85]:
dd.to_csv("data/data_dictionary_v4.csv", index=False)

In [95]:
dd[dd['Renamed Variable'].isin(df.columns)][['Renamed Variable', 'Survey Question']].values

array([['state_fips_code', 'State FIPS Code'],
       ['sex', 'Sex of Respondent'],
       ['age_group', 'Fourteen-level age category'],
       ['race_ethnicity', 'Five-level race/ethnicity category'],
       ['education_level', 'Level of education completed'],
       ['income_category', 'Income categories'],
       ['marital_status', 'Are you: (marital status)'],
       ['employment_status', 'Are you currently…?'],
       ['uses_tobacco',
        'Do you currently use chewing tobacco, snuff, or snus every day, some days, or not at all?  (Snus (Swedish for snuff) is a moist smokeless tobacco, usually sold in small pouches that are placed under the lip against the gum.)'],
       ['uses_e_cigarettes',
        'Would you say you have never used e-cigarettes or other electronic vaping products in your entire life or now use them every day, use them some days, or used them in the past but do not currently use them at all?'],
       ['days_drinking_alcohol',
        'During the past 30 days

### Archive

In [3]:
import pandas as pd

pd.read_csv("data/data_dictionary.csv")[['Data Element Name', 'Description']].values

array([['CVDINFR4', 'Ever Diagnosed with Heart Attack'],
       ['CVDCRHD4',
        'Ever Diagnosed with Angina or Coronary Heart Disease'],
       ['_MICHD', 'Ever had CHD or MI'],
       ['_STATE', 'State FIPS Code'],
       ['SEXVAR', 'Sex of Respondent'],
       ['_AGEG5YR',
        'Reported age in five-year age categories calculated variable'],
       ['_RACEGR4', 'Computed Five level race/ethnicity category.'],
       ['_EDUCAG', 'Computed level of education completed categories'],
       ['_INCOMG1', 'Computed income categories'],
       ['MARITAL', 'Marital Status'],
       ['EMPLOY1', 'Employment Status'],
       ['SMOKE100', 'Smoked at Least 100 Cigarettes'],
       ['USENOW3', 'Use of Smokeless Tobacco Products'],
       ['ECIGNOW2',
        'Do you now use e-cigarettes, or vaping products every day, some days, or not at all?'],
       ['ALCDAY4', 'Days in past 30 had alcoholic beverage'],
       ['EXERANY2', 'Exercise in Past 30 Days'],
       ['SLEPTIM1', 'How Much Time 

       ['CVDINFR4', 'Ever Diagnosed with Heart Attack'],
       ['CVDCRHD4', 'Ever Diagnosed with Angina or Coronary Heart Disease'],
       ['_MICHD', 'Ever had CHD or MI'],
       ['_STATE', 'State FIPS Code'],
       ['SEXVAR', 'Sex of Respondent'],
       ['_AGEG5YR', 'Reported age in five-year age categories calculated variable'],
       ['_RACEGR4', 'Computed Five level race/ethnicity category.'],
       ['_EDUCAG', 'Computed level of education completed categories'],
       ['_INCOMG1', 'Computed income categories'],
       ['MARITAL', 'Marital Status'],
       ['EMPLOY1', 'Employment Status'],
       ['SMOKE100', 'Smoked at Least 100 Cigarettes'],
       ['USENOW3', 'Use of Smokeless Tobacco Products'],
       ['ECIGNOW2', 'Do you now use e-cigarettes, or vaping products every day, some days, or not at all?'],
       ['ALCDAY4', 'Days in past 30 had alcoholic beverage'],
       ['EXERANY2', 'Exercise in Past 30 Days'],
       ['SLEPTIM1', 'How Much Time Do You Sleep'],
       ['GENHLTH', 'General Health'],
       ['PHYSHLTH', 'Number of Days Physical Health Not Good'],
       ['MENTHLTH', 'Number of Days Mental Health Not Good'],
       ['DIABETE4', '(Ever told) you had diabetes'],
       ['HAVARTH4', 'Told Had Arthritis'],
       ['CHCCOPD3', 'Ever told you had C.O.P.D. emphysema or chronic bronchitis?'],
       ['ADDEPEV3', '(Ever told) you had a depressive disorder'],
       ['ASTHMA3', 'Ever Told Had Asthma'],
       ['WEIGHT2', 'Reported Weight in Pounds'],
       ['HEIGHT3', 'Reported Height in Feet and Inches'],
       ['_BMI5', 'Computed body mass index'],
       ['_BMI5CAT', 'Computed body mass index categories'],
       ['PRIMINSR', 'What is Primary Source of Health Insurance?'],
       ['MEDCOST1', 'Could Not Afford To See Doctor'],
       ['CHECKUP1', 'Length of time since last routine checkup'],
       ['COVIDPOS','Have you ever been told you tested positive for COVID 19?'],
       ['FLUSHOT7', 'Adult flu shot/spray past 12 mos'],
       ['PNEUVAC4', 'Pneumonia shot ever'],
       ['TETANUS1', 'Received Tetanus Shot Since 2005?']

In [6]:
pd.read_csv("data/dataset.csv").columns.shape

(37,)